In [68]:
import json
import requests

In [69]:
endpoint_legacy = 'https://wikimedia.org/api/rest_v1/metrics/legacy/pagecounts/aggregate/{project}/{access-site}/{granularity}/{start}/{end}'
endpoint_pageviews = 'https://wikimedia.org/api/rest_v1/metrics/pageviews/aggregate/{project}/{access}/{agent}/{granularity}/{start}/{end}'


In [70]:
headers = {
    'User-Agent': 'https://github.com/kevSweet',
    'From': 'ksweet1@uw.edu'
}

# dump dictionary to file
def write_to_json(data, fn):
    with open(fn, "w") as outfile:
        json.dump(data, outfile)
        
def api_call(endpoint,parameters):
    call = requests.get(endpoint.format(**parameters), headers=headers)
    response = call.json()
    
    return response

In [71]:
# see: https://wikimedia.org/api/rest_v1/#!/Pageviews_data/get_metrics_pageviews_aggregate_project_access_agent_granularity_start_end
params_legacy = {"project" : "en.wikipedia.org",
                 "access-site" : "desktop-site",
                 "granularity" : "monthly",
                 "start" : "2001010100",
                # for end use 1st day of month following final month of data
                 "end" : "2018100100"
                    }

# pagecounts dump

# desktop
monthly_legacy = api_call(endpoint_legacy, params_legacy)
first_date = monthly_legacy['items'][0]['timestamp']
last_date = monthly_legacy['items'][-1]['timestamp']
legacy_desktopfn = 'pagecounts_desktop-site_' + first_date + '-' + last_date + '.json'
write_to_json(monthly_legacy, legacy_desktopfn)

# mobile
params_legacy['access-site'] = "mobile-site"
monthly_legacy = api_call(endpoint_legacy, params_legacy)
first_date = monthly_legacy['items'][0]['timestamp']
last_date = monthly_legacy['items'][-1]['timestamp']
legacy_mobilefn = 'pagecounts_mobile-site_' + first_date + '-' + last_date + '.json'
write_to_json(monthly_legacy, legacy_mobilefn)

In [72]:

params_pageviews = {"project" : "en.wikipedia.org",
                    "access" : "desktop",
                    "agent" : "user",
                    "granularity" : "monthly",
                    "start" : "2001010100",
                    # for end use 1st day of month following final month of data
                    "end" : '2021110100'
                        }

# pageviews dump

# desktop
monthly_pageviews = api_call(endpoint_pageviews, params_pageviews)
first_date = monthly_pageviews['items'][0]['timestamp']
last_date = monthly_pageviews['items'][-1]['timestamp']
pageviews_desktopfn = 'pageviews_desktop-site_'+ first_date + '-' + last_date + '.json'
write_to_json(monthly_pageviews, pageviews_desktopfn)

# mobile app
params_pageviews['access'] = "mobile-app"
monthly_pageviews = api_call(endpoint_pageviews, params_pageviews)
first_date = monthly_pageviews['items'][0]['timestamp']
last_date = monthly_pageviews['items'][-1]['timestamp']
pageviews_mobileappfn = 'pageviews_mobile-app_'+ first_date + '-' + last_date + '.json'
write_to_json(monthly_pageviews, pageviews_mobileappfn)

# mobile web
params_pageviews['access'] = "mobile-web"
monthly_pageviews = api_call(endpoint_pageviews, params_pageviews)
first_date = monthly_pageviews['items'][0]['timestamp']
last_date = monthly_pageviews['items'][-1]['timestamp']
pageviews_mobilewebfn = 'pageviews_mobile-web_'+ first_date + '-' + last_date + '.json'
write_to_json(monthly_pageviews, pageviews_mobilewebfn)